# Imputing summary statistics from LD reference panel

This notebook take a list of LD reference files and a list of sumstat files from various association studies ...

## Input

1. **FIXME we need to make input as a bed file with chrom, start and end** A tab delimated table describing the path where LD per region stored, can be generated using the ld_per_region_plink step of the genotype processing module.

```
#id     dir
chr17_60570445_65149278 /mnt/vast/hpc/csg/molecular_phenotype_calling/LD/output_npz_2/1300_hg38_EUR_LD_blocks_npz_files/ROSMAP_NIA_WGS.leftnorm.filtered.filtered.chr17_60570445_65149278.flt16.npz
```

2. A tab delimated table describing path where summary stat per chromosome stored, can be generated using the yml_generator module before the qced sumstat are generated. **FIXME: If the chrom name is zero that means the data is genome-wide**
```
hs3163@csglogin:/mnt/vast/hpc/csg/xqtl_workflow_testing/susie_rss$ cat /mnt/vast/hpc/csg/xqtl_workflow_testing/ADGWAS/data_intergration/ADGWAS2022/qced_sumstat_list.txt
#chr    ADGWAS_Bellenguez_2022
1       /mnt/vast/hpc/csg/xqtl_workflow_testing/ADGWAS/data_intergration/ADGWAS2022/ADGWAS_Bellenguez_2022.1/ADGWAS2022.chr1.sumstat.tsv
2       /mnt/vast/hpc/csg/xqtl_workflow_testing/ADGWAS/data_intergration/ADGWAS2022/ADGWAS_Bellenguez_2022.2/ADGWAS2022.chr2.sumstat.tsv
3       /mnt/vast/hpc/csg/xqtl_workflow_testing/ADGWAS/data_intergration/ADGWAS2022/ADGWAS_Bellenguez_2022.3/ADGWAS2022.chr3.sumstat.tsv
4       /mnt/vast/hpc/csg/xqtl_workflow_testing/ADGWAS/data_intergration/ADGWAS2022/ADGWAS_Bellenguez_2022.4/ADGWAS2022.chr4.sumstat.tsv
5       /mnt/vast/hpc/csg/xqtl_workflow_testing/ADGWAS/data_intergration/ADGWAS2022/ADGWAS_Bellenguez_2022.5/ADGWAS2022.chr5.sumstat.tsv
```

3. Regions we want to analyze in the format `chr:start-end`. Can be multiple of these. If not specified we will use the regions in the LD data list

## Output

1. A RDS file containing the output susie object, the name of all variants that went through the analysis, the z score , and the LD used for the analysis.
2. A sumstat file with additional column containing the slalom results.

## MWE

In [ ]:
sos run pipeline/SuSiE_RSS.ipynb SuSiE_RSS \
    --ld-data test.ld.list \
    --sumstats /mnt/vast/hpc/csg/xqtl_workflow_testing/ADGWAS/data_intergration/ADGWAS2022/qced_sumstat_list.txt \
    --container containers/stephenslab.sif --impute --cwd output_impute_2

In [ ]:
[global]
parameter: cwd = path("output")
# getting the overlapped input
parameter: ld_data = path
parameter: sumstats = paths
import pandas as pd
parameter: container = ""
import re
parameter: entrypoint= ('micromamba run -a "" -n' + ' ' + re.sub(r'(_apptainer:latest|_docker:latest|\.sif)$', '', container.split('/')[-1])) if container else ""
# For cluster jobs, number commands to run per job
parameter: job_size = 1
# Wall clock time expected
parameter: walltime = "5h"
# Memory expected
parameter: mem = "16G"
# Number of threads
parameter: numThreads = 3

parameter: lead_idx_choice = "pvalue"
parameter: abf_prior_variance = 0.4
parameter: nlog10p_dentist_s_threshold = 4
parameter: r2_threshold = 0.6
parameter: n = 0
parameter: max_iter = 1000
parameter: impute = True # Whether to impute the sumstat for all the snp in LD but not in sumstat.

In [ ]:
[get_analysis_regions: shared = "regional_data"]
# This will pair the LD matrix blocks with each of the input summary stats

LD_list = pd.read_csv(LD_list,sep="\t")
sumstat_list = pd.read_csv(sumstats,sep="\t")
LD_list["#chr"] = [x[0].replace("chr", "") for x in  LD_list["#id"].str.split("_") ]
sumstat_list["#chr"] = [str(x).replace("chr", "") for x in  sumstat_list["#chr"] ]
input_inv = LD_list.merge(sumstat_list)
input_list = input_inv.iloc[:,[1,3]].values.tolist()

In [ ]:
[SuSiE_RSS_1]
parameter: L = 10
parameter: max_L = 1000

depends: sos_variable("regional_data")

meta_info = regional_data['meta_info']
input: regional_data["data"], group_by = 2, group_with = "meta_info"
# name = f'{_input[0]:b}'.split(".")[-3]
output: f'{cwd:a}/{_input[1]:bn}.{name}.unisusie_rss.fit.rds',
        f'{cwd:a}/{_input[1]:bn}.{name}.unisusie_rss.ss_qced.tsv'    
task: trunk_workers = 1, trunk_size = job_size, walltime = walltime, mem = mem, cores = numThreads, tags = f'{step_name}_{_output[0]:bn}'
R: expand = '${ }', stdout = f"{_output[0]:nn}.stdout", stderr = f"{_output[0]:nn}.stderr", container = container, entrypoint = entrypoint
  
    ## Step 1: Load summary stats and LD data using the function in pecotmr::LD.R

    ## Step 2: basic QC between LD and summary stats --- to correct allele flipping mainly
  
    ## Step 3: Perform SuSiE RSS
  
    ## Output is RDS file including 1) fine-mapping results and 2) the summary stats after allele flipping QC as well as the SuSiE RSS based QC